In [2]:
import pandas as pd

In [3]:
import numpy as np
import itertools
import re
import ordered_set
from ordered_set import OrderedSet

In [4]:
class itemset:
    def __init__(self, items, support):
        self.items = items
        self.support = support
        
    def to_string(self):
        return "Items:" + str(self.items) + "\n Support:" + str(self.support)


In [434]:
class Rules:
    def __init__(self, x, y):
        self.x=x
        self.y=y
    
    def support_count(self, l):
        ls=[]
        for se in sorted(l):
            f_df = df[se]
            if(isinstance(f_df, pd.Series)):
                ls.append(itemset(OrderedSet([se]), sum(f_df)))
            else:
                ls.append(itemset(se, sum(f_df.all(axis=1))))
        return ls
        
    def confidence_x(self):
        
        xs=(self.x[0])
        ys=(self.y[0])
        return (self.support_count([xs | ys])[0].support) /(self.support_count(self.x)[0].support)
    
    def confidence_y(self):
        
        xs=(self.x[0])
        ys=(self.y[0])
        return (self.support_count([xs | ys])[0].support) /(self.support_count(self.y)[0].support)

In [435]:
data = {"customer_id": [1,1,2,2,3,3,4,4,5], "transaction_id": [1,2,3,4,5,6,7,8,9], 
        "items": [["Milk", "Bread", "Butter"], ["Bread", "Cheese"], ["Bread", "Jam"],
                  ["Milk", "Bread", "Cheese"], ["Milk", "Jam"], ["Bread", "Jam"], ["Milk", "Jam"], 
                  ["Milk", "Bread", "Jam", "Butter"], ["Milk", "Bread", "Jam"]]}
        

In [436]:
df=pd.DataFrame(data)

In [437]:
df

,customer_id,transaction_id,items
0,1,1,"[Milk, Bread, Butter]"
1,1,2,"[Bread, Cheese]"
2,2,3,"[Bread, Jam]"
3,2,4,"[Milk, Bread, Cheese]"
4,3,5,"[Milk, Jam]"
5,3,6,"[Bread, Jam]"
6,4,7,"[Milk, Jam]"
7,4,8,"[Milk, Bread, Jam, Butter]"
8,5,9,"[Milk, Bread, Jam]"


In [438]:
#Index - pass lis
df = df[['transaction_id', 'items']]

In [439]:
for idx, row in df.iterrows():
    for item in row['items']:
        df.loc[idx, item] = True

In [440]:
df.fillna(False, inplace=True)

In [441]:
df.drop('items', axis=1, inplace=True)
df.set_index(['transaction_id'], inplace=True)

In [442]:
df

,Milk,Bread,Butter,Cheese,Jam
transaction_id,,,,,
1,True,True,True,False,False
2,False,True,False,True,False
3,False,True,False,False,True
4,True,True,False,True,False
5,True,False,False,False,True
6,False,True,False,False,True
7,True,False,False,False,True
8,True,True,True,False,True
9,True,True,False,False,True


In [443]:
class Apriori:
    
    def __init__(self, df):
        self.df = df
        
    def support_count(self, l):
        ls=[]
        for se in sorted(l):
            f_df = df[se]
            if(isinstance(f_df, pd.Series)):
                ls.append(itemset(OrderedSet([se]), sum(f_df)))
            else:
                ls.append(itemset(se, sum(f_df.all(axis=1))))
        return ls
    
    def create_k_set(self,iters, k):
        ls = []
        for i in range(len(iters)):
            for j in range(len(iters) -1) :
                s1 = iters[i].items
                s2 = iters[j+1].items
                if (s1[0:k-2] == s2[0:k-2]) and (len(s1) ==  len(s2)):
                    s = s1 | s2
                    s = sorted(s)
                    s = OrderedSet(s)
                    if s not in ls and(s1 != s2):
                        ls.append(s)
        return ls
    
    def filter_minsup(self, ls, minsup):
        new_ls = []
        stop_algo = True
        for items in ls:
            if(items.support >= minsup):
                new_ls.append(items)
                if items.support != minsup:
                    stop_algo = False
        return new_ls, stop_algo
    
    def create_x(self, k2, i):     
        xs = {}
        idx =0 
        while(i> 1):
            for xa in k2:
                if i > 1:
                    xs[idx] = list(itertools.combinations(xa.items, i))
                    idx+=1
            i-=1
        return xs
    
    def create_y(self, k2, xs):
        ys = {}
        for idx, itemx in xs.items():
            temp_y = []
            all_items= set(list(itertools.chain(*itemx)))
            for yi in range(len(itemx)):
                
                y = OrderedSet(itemx[yi]).symmetric_difference(all_items)
                print(y)
                temp_y.append(y)
            ys[idx] = temp_y
        return ys
        
    def create_rules(self, k2, k):
        xs = self.create_x(k2, k)
        ys = self.create_y(k2, xs)
        
        rs = []
        for idx, item in  xs.items():
            for i in range(len(item)):
                yi = ys[idx][i]
                xi = item[i]
                print("yi",yi)
                print("xi", xi)
                rs.append(Rules([OrderedSet(xi)], [(yi)]))
        
        return rs
        
    
    def apriori(self, minssup):
        k_set=list(df.columns)
        k=2
        l_k_set = 0
        while True:
            ck_set = self.support_count(k_set)
            t_k_set, stop_algo = self.filter_minsup(ck_set, minssup)
            if stop_algo: break
            k_set = self.create_k_set(t_k_set, k)
            k+=1
            l_k_set+=1
        final_k_set = t_k_set if len(t_k_set) > 0 else k_set
        return self.create_rules(final_k_set, l_k_set)
    
    

    

In [444]:
test = Apriori(df)

In [450]:
rs =test.apriori(1)

OrderedSet(['Milk'])
OrderedSet(['Jam'])
OrderedSet(['Butter'])
OrderedSet(['Bread'])
OrderedSet(['Milk', 'Jam'])
OrderedSet(['Milk', 'Butter'])
OrderedSet(['Jam', 'Butter'])
OrderedSet(['Bread', 'Milk'])
OrderedSet(['Bread', 'Jam'])
OrderedSet(['Bread', 'Butter'])
yi OrderedSet(['Milk'])
xi ('Bread', 'Butter', 'Jam')
yi OrderedSet(['Jam'])
xi ('Bread', 'Butter', 'Milk')
yi OrderedSet(['Butter'])
xi ('Bread', 'Jam', 'Milk')
yi OrderedSet(['Bread'])
xi ('Butter', 'Jam', 'Milk')
yi OrderedSet(['Milk', 'Jam'])
xi ('Bread', 'Butter')
yi OrderedSet(['Milk', 'Butter'])
xi ('Bread', 'Jam')
yi OrderedSet(['Jam', 'Butter'])
xi ('Bread', 'Milk')
yi OrderedSet(['Bread', 'Milk'])
xi ('Butter', 'Jam')
yi OrderedSet(['Bread', 'Jam'])
xi ('Butter', 'Milk')
yi OrderedSet(['Bread', 'Butter'])
xi ('Jam', 'Milk')


In [451]:
for r in rs:
    print(str(r.y) + " -> " + str(r.x) + str(r.confidence_y()))
print("=======")
for r in rs:
    print(str(r.x) + " -> " + str(r.y) + str(r.confidence_x()))

[OrderedSet(['Milk'])] -> [OrderedSet(['Bread', 'Butter', 'Jam'])]0.16666666666666666
[OrderedSet(['Jam'])] -> [OrderedSet(['Bread', 'Butter', 'Milk'])]0.16666666666666666
[OrderedSet(['Butter'])] -> [OrderedSet(['Bread', 'Jam', 'Milk'])]0.5
[OrderedSet(['Bread'])] -> [OrderedSet(['Butter', 'Jam', 'Milk'])]0.14285714285714285
[OrderedSet(['Milk', 'Jam'])] -> [OrderedSet(['Bread', 'Butter'])]0.25
[OrderedSet(['Milk', 'Butter'])] -> [OrderedSet(['Bread', 'Jam'])]0.5
[OrderedSet(['Jam', 'Butter'])] -> [OrderedSet(['Bread', 'Milk'])]1.0
[OrderedSet(['Bread', 'Milk'])] -> [OrderedSet(['Butter', 'Jam'])]0.25
[OrderedSet(['Bread', 'Jam'])] -> [OrderedSet(['Butter', 'Milk'])]0.25
[OrderedSet(['Bread', 'Butter'])] -> [OrderedSet(['Jam', 'Milk'])]0.5
[OrderedSet(['Bread', 'Butter', 'Jam'])] -> [OrderedSet(['Milk'])]1.0
[OrderedSet(['Bread', 'Butter', 'Milk'])] -> [OrderedSet(['Jam'])]0.5
[OrderedSet(['Bread', 'Jam', 'Milk'])] -> [OrderedSet(['Butter'])]0.5
[OrderedSet(['Butter', 'Jam', 'Milk'])]

In [234]:
print(foo())

NameError: name 'foo' is not defined

In [16]:
initial_set=list(df.columns)

In [17]:
da= support_count(initial_set)

In [ ]:
n_s1 = filter_minsup(da, 2)

In [19]:
t=create_k_set(da, 2)

In [21]:
s1 = support_count(t)

In [23]:
n_s1 = filter_minsup(s1, 2)

In [24]:
k1=create_k_set(n_s1, 3)

In [26]:
k1 = support_count(k1)

In [66]:
k2 = filter_minsup(k1, 2)

In [92]:
k2[0].items

OrderedSet(['Bread', 'Butter', 'Milk'])

In [96]:
test=[itemset(OrderedSet(['Bread', "Butter", "Milk", "Jam"]),0)]

In [97]:
test

In [98]:
xs = []
i=3
while(i> 1):
    for xa in test:
        if i > 1:
            xs.append(list(itertools.combinations(xa.items, i)))
    i-=1

In [99]:
xs

[[('Bread', 'Butter', 'Milk'),
  ('Bread', 'Butter', 'Jam'),
  ('Bread', 'Milk', 'Jam'),
  ('Butter', 'Milk', 'Jam')],
 [('Bread', 'Butter'),
  ('Bread', 'Milk'),
  ('Bread', 'Jam'),
  ('Butter', 'Milk'),
  ('Butter', 'Jam'),
  ('Milk', 'Jam')]]

In [115]:
ys =[]
for idx, val in enumerate(test):
    for yis in xs:
        for yi in yis:
            y = set(yi).symmetric_difference(val.items)
            ys.append(y)
       

('Bread', 'Butter', 'Milk')
('Bread', 'Butter', 'Jam')
('Bread', 'Milk', 'Jam')
('Butter', 'Milk', 'Jam')
('Bread', 'Butter')
('Bread', 'Milk')
('Bread', 'Jam')
('Butter', 'Milk')
('Butter', 'Jam')
('Milk', 'Jam')


In [116]:
ys

[{'Jam'},
 {'Milk'},
 {'Butter'},
 {'Bread'},
 {'Jam', 'Milk'},
 {'Butter', 'Jam'},
 {'Butter', 'Milk'},
 {'Bread', 'Jam'},
 {'Bread', 'Milk'},
 {'Bread', 'Butter'}]

In [85]:
rs = []
i = 0 
for idx, item in  enumerate(xs):
    for idxs, itemx in enumerate(item):   
        rs.append(Rules([OrderedSet(itemx)], [OrderedSet(ys[i])]))
        i+=1
        

In [86]:
for r in rs:
    print(r.x)
    print(r.y)

[OrderedSet(['Bread', 'Butter'])]
[OrderedSet(['Milk'])]
[OrderedSet(['Bread', 'Milk'])]
[OrderedSet(['Butter'])]
[OrderedSet(['Butter', 'Milk'])]
[OrderedSet(['Bread'])]
[OrderedSet(['Bread', 'Jam'])]
[OrderedSet(['Milk'])]
[OrderedSet(['Bread', 'Milk'])]
[OrderedSet(['Jam'])]
[OrderedSet(['Jam', 'Milk'])]
[OrderedSet(['Bread'])]


In [87]:
[(rs[0].x)]

[[OrderedSet(['Bread', 'Butter'])]]

In [88]:
t =support_count(list(rs[0].y))

In [89]:
print(rs[0].x)
rs[0].y[0] | rs[0].x[0]

[OrderedSet(['Bread', 'Butter'])]


OrderedSet(['Milk', 'Bread', 'Butter'])

In [90]:
for r in rs:
    # print(str(r.x) + " -> " + str(r.y) + str(r.confidence_x()))
    print(str(r.y) + " -> " + str(r.x) + str(r.confidence_y()))
    # print(r.confidence_y())

[OrderedSet(['Milk'])] -> [OrderedSet(['Bread', 'Butter'])]0.3333333333333333
[OrderedSet(['Butter'])] -> [OrderedSet(['Bread', 'Milk'])]1.0
[OrderedSet(['Bread'])] -> [OrderedSet(['Butter', 'Milk'])]0.2857142857142857
[OrderedSet(['Milk'])] -> [OrderedSet(['Bread', 'Jam'])]0.3333333333333333
[OrderedSet(['Jam'])] -> [OrderedSet(['Bread', 'Milk'])]0.3333333333333333
[OrderedSet(['Bread'])] -> [OrderedSet(['Jam', 'Milk'])]0.2857142857142857
